In [3]:
import cv2
import os
import json

# パラメータファイルの読み込み
with open('parameters.json', 'r') as f:
    params = json.load(f)

# 入力動画フォルダと出力画像フォルダの設定
input_folder = params['input_folder']
output_folder = params['output_folder']

# 出力フォルダが存在しない場合は作成
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 動画ファイルの処理
for video_file in os.listdir(input_folder):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # 対応する動画形式を追加
        video_path = os.path.join(input_folder, video_file)
        cap = cv2.VideoCapture(video_path)
        
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            if frame_count % params['frame_interval'] == 0:
                output_path = os.path.join(output_folder, f"{video_file[:-4]}_{frame_count:06d}.jpg")
                cv2.imwrite(output_path, frame)
            
            frame_count += 1
        
        cap.release()

print("処理が完了しました。")

処理が完了しました。
